In [1]:
# --- ПЕРВАЯ ЯЧЕЙКА: ОСНОВНАЯ НАСТРОЙКА И ЗАГРУЗКА ---
import torch
import torchaudio
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor
import time
import psutil
from datasets import load_dataset, Audio
import evaluate
import os
import gc
from tqdm.auto import tqdm
import copy # Для копирования модели
import numpy as np # Для k-means и работы с весами
from sklearn.cluster import KMeans # Для кластеризации

# --- 1. Конфигурация ---
MODEL_ID = "openai/whisper-large-v3"
DATASET_ID = "mozilla-foundation/common_voice_16_1"
DATASET_NAME = "ru"
DATASET_SPLIT = "test" # Для оценки качества
# CALIBRATION_SPLIT = "train" # Не используется в этой ячейке, но может быть для других
NUM_SAMPLES_FOR_QUALITY_TEST = 50
NUM_SAMPLES_FOR_TIME_TEST = 10
NUM_WARMUP_RUNS = 3
# NUM_CALIBRATION_SAMPLES = 20 # Не используется здесь
TARGET_SAMPLE_RATE = 16000

# --- 2. Определение устройства и типа данных ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Для базовой модели и сравнений обычно используется FP16 на GPU или FP32 на CPU
# В коде кластеризации мы будем загружать модель в FP32 на CPU для k-means,
# а затем переносить на целевое устройство и в целевой dtype (который может быть FP16 на GPU)
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
print(f"Основное устройство для оценки: {device}, основной dtype: {torch_dtype}")

# --- 3. Загрузка ПРОЦЕССОРА (нужен для функций оценки) ---
print(f"Загрузка процессора для {MODEL_ID}...")
processor = AutoProcessor.from_pretrained(MODEL_ID)
print("Процессор загружен.")

# --- 4. Загрузка МЕТРИК ОЦЕНКИ (нужны для функций оценки) ---
wer_metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")
print("Метрики WER и CER загружены.")

# --- 5. Подготовка FORCED_DECODER_IDS (нужны для функций оценки Whisper) ---
forced_decoder_ids = processor.get_decoder_prompt_ids(language="russian", task="transcribe")
print("forced_decoder_ids подготовлены.")

# --- 6. Определение ФУНКЦИЙ ОЦЕНКИ КАЧЕСТВА И ПРОИЗВОДИТЕЛЬНОСТИ ---
# Эти функции будут использоваться для оценки как оригинальной, так и кластеризованной модели.

def evaluate_quality_whisper(model_to_eval, processor_to_use, dataset_to_eval, num_samples, device_to_use, dtype_to_use, forced_decoder_ids_for_eval):
    model_to_eval.to(device_to_use, dtype=dtype_to_use) # Убедимся, что модель на нужном устройстве и в нужном dtype
    model_to_eval.eval()
    _predictions = []
    _references = []
    
    for i in tqdm(range(min(num_samples, len(dataset_to_eval))), desc=f"Оценка качества ({str(device_to_use)}, {str(dtype_to_use)})"):
        sample = dataset_to_eval[i]
        reference_text = sample["sentence"]
        if not reference_text or reference_text.strip() == "":
            continue

        raw_audio = sample["audio"]["array"]
        sampling_rate = sample["audio"]["sampling_rate"]

        if len(raw_audio) == 0:
            _predictions.append("")
            _references.append(reference_text.lower())
            continue
        
        input_features = processor_to_use(raw_audio, sampling_rate=sampling_rate, return_tensors="pt").input_features
        input_features = input_features.to(device_to_use, dtype=dtype_to_use)

        with torch.no_grad():
            predicted_ids = model_to_eval.generate(input_features, forced_decoder_ids=forced_decoder_ids_for_eval)
        
        predicted_text = processor_to_use.batch_decode(predicted_ids, skip_special_tokens=True)[0]
        _predictions.append(predicted_text.strip().lower())
        _references.append(reference_text.lower())
        del input_features, predicted_ids
        if device_to_use.type == 'cuda':
            torch.cuda.empty_cache()

    _wer, _cer = "N/A", "N/A"
    if _predictions and _references:
        _wer = wer_metric.compute(predictions=_predictions, references=_references)
        _cer = cer_metric.compute(predictions=_predictions, references=_references)
    return _wer, _cer, _predictions, _references

def evaluate_performance_whisper(model_to_eval, processor_to_use, audio_list_for_timing, num_warmup, num_test, device_to_use, dtype_to_use, forced_decoder_ids_for_eval):
    model_to_eval.to(device_to_use, dtype=dtype_to_use) # Убедимся, что модель на нужном устройстве и в нужном dtype
    model_to_eval.eval()
    _times = []
    _vram_usage_mb, _ram_usage_mb = "N/A", "N/A"
    
    desc_perf = f"Замеры производительности ({str(device_to_use)}, {str(dtype_to_use)})"

    # Прогрев
    # print(f"Прогрев ({desc_perf})...")
    for i in range(min(num_warmup, len(audio_list_for_timing))):
        raw_audio = audio_list_for_timing[i]
        input_features = processor_to_use(raw_audio, sampling_rate=TARGET_SAMPLE_RATE, return_tensors="pt").input_features.to(device_to_use, dtype=dtype_to_use)
        with torch.no_grad():
            _ = model_to_eval.generate(input_features, forced_decoder_ids=forced_decoder_ids_for_eval)
        del input_features
        if device_to_use.type == 'cuda':
            torch.cuda.synchronize() # Синхронизация после каждой операции на GPU в прогреве
            
    if device_to_use.type == 'cuda':
        torch.cuda.reset_peak_memory_stats(device_to_use) # Сбрасываем память ПОСЛЕ прогрева
        torch.cuda.synchronize() # Убедимся, что reset завершен

    ps_process = psutil.Process(os.getpid())
    # ram_before_mb = ps_process.memory_info().rss / (1024 * 1024) # RAM перед замерами

    for i in tqdm(range(min(num_test, len(audio_list_for_timing))), desc=desc_perf):
        raw_audio = audio_list_for_timing[i]
        input_features = processor_to_use(raw_audio, sampling_rate=TARGET_SAMPLE_RATE, return_tensors="pt").input_features.to(device_to_use, dtype=dtype_to_use)
        
        if device_to_use.type == 'cuda':
            torch.cuda.synchronize() # Синхронизация ПЕРЕД стартом таймера
        start_time = time.perf_counter()
        
        with torch.no_grad():
            _ = model_to_eval.generate(input_features, forced_decoder_ids=forced_decoder_ids_for_eval)
        
        if device_to_use.type == 'cuda':
            torch.cuda.synchronize() # Синхронизация ПОСЛЕ завершения операций на GPU
        end_time = time.perf_counter()
        _times.append((end_time - start_time) * 1000) # ms
        
        if i == 0 and device_to_use.type == 'cuda': # VRAM на первом реальном замере
            _vram_usage_mb = torch.cuda.max_memory_allocated(device_to_use) / (1024 * 1024)
        del input_features
        if device_to_use.type == 'cuda':
            torch.cuda.empty_cache() # Очищаем кэш после каждого инференса

    _avg_time_ms = sum(_times) / len(_times) if _times else "N/A"
    _ram_usage_mb = ps_process.memory_info().rss / (1024 * 1024) # RAM после всех замеров
    
    return _avg_time_ms, _vram_usage_mb if device_to_use.type == 'cuda' else "N/A", _ram_usage_mb

print("Функции оценки качества и производительности определены.")

# --- 7. ЗАГРУЗКА ДАТАСЕТОВ (чтобы они были доступны для последующих ячеек) ---
print(f"Загрузка датасета {DATASET_ID} для оценки качества ({DATASET_SPLIT} split)...")
quality_dataset = load_dataset(DATASET_ID, DATASET_NAME, split=f"{DATASET_SPLIT}[:{NUM_SAMPLES_FOR_QUALITY_TEST}]", trust_remote_code=True)
quality_dataset = quality_dataset.cast_column("audio", Audio(sampling_rate=TARGET_SAMPLE_RATE))
print(f"Загружено {len(quality_dataset)} сэмплов для оценки качества.")

print(f"Загрузка датасета {DATASET_ID} для замеров производительности ({DATASET_SPLIT} split)...")
timing_dataset_raw_obj = load_dataset(DATASET_ID, DATASET_NAME, split=f"{DATASET_SPLIT}[:{NUM_SAMPLES_FOR_TIME_TEST + NUM_WARMUP_RUNS}]", trust_remote_code=True)
timing_dataset_raw_obj = timing_dataset_raw_obj.cast_column("audio", Audio(sampling_rate=TARGET_SAMPLE_RATE))
raw_audio_list_for_timing = [sample["audio"]["array"] for sample in timing_dataset_raw_obj]
print(f"Загружено {len(raw_audio_list_for_timing)} аудиодорожек для замеров производительности.")

# --- (ОПЦИОНАЛЬНО) Загрузка базовой модели для сравнения, если нужно ---
# Если вы хотите сравнить кластеризованную модель с нетронутой FP16/FP32 моделью в этом же ноутбуке,
# вы можете загрузить ее здесь и сохранить.
# В коде кластеризации мы загружаем модель заново для чистоты эксперимента,
# но для финальной таблицы вам понадобятся baseline-метрики.

# print(f"Загрузка базовой модели {MODEL_ID} для справки...")
# baseline_model = AutoModelForSpeechSeq2Seq.from_pretrained(
#     MODEL_ID,
#     torch_dtype=torch_dtype,
#     low_cpu_mem_usage=True,
#     use_safetensors=True
# ).to(device)
# baseline_model.eval()
# print("Базовая модель загружена (если нужна для прямого сравнения в этом ноутбуке).")

print("\n--- Первая ячейка: настройка завершена. Можно переходить к кластеризации. ---")

/home/ubuntu/miniconda3/envs/model-compression/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Основное устройство для оценки: cuda, основной dtype: torch.float16
Загрузка процессора для openai/whisper-large-v3...
Процессор загружен.
Метрики WER и CER загружены.
forced_decoder_ids подготовлены.
Функции оценки качества и производительности определены.
Загрузка датасета mozilla-foundation/common_voice_16_1 для оценки качества (test split)...
Загружено 50 сэмплов для оценки качества.
Загрузка датасета mozilla-foundation/common_voice_16_1 для замеров производительности (test split)...
Загружено 13 аудиодорожек для замеров производительности.

--- Первая ячейка: настройка завершена. Можно переходить к кластеризации. ---


In [2]:
# --- ЯЧЕЙКА ДЛЯ КЛАСТЕРИЗАЦИИ ВЕСОВ (K-MEANS) ---
# Предполагается, что переменные из ПЕРВОЙ ячейки (конфигурация, пути, processor, etc.) доступны.
# Функции evaluate_quality_whisper и evaluate_performance_whisper также должны быть определены ранее.

from sklearn.cluster import KMeans
import numpy as np

print(f"--- Кластеризация весов K-Means для {MODEL_ID} ---")

# 1. Параметры кластеризации
N_CLUSTERS = 32 # Количество кластеров для весов каждого слоя. Можете поэкспериментировать.
# Будем кластеризовать только линейные слои (самые большие по количеству весов в трансформерах)
LAYERS_TO_CLUSTER = [torch.nn.Linear]

# 2. Загрузка "чистой" модели или создание глубокой копии
# Важно работать с копией, чтобы не изменять оригинальную FP16 модель перманентно.
# Используем устройство и dtype из первой ячейки для консистентности.
cluster_device = device
cluster_dtype = torch_dtype

print(f"Создание копии модели {MODEL_ID} для кластеризации на {cluster_device} с dtype {cluster_dtype}...")
# Сначала загрузим на CPU в FP32, чтобы избежать проблем с k-means на GPU/FP16, затем скопируем и перенесем.
model_fp32_cpu_for_clustering = AutoModelForSpeechSeq2Seq.from_pretrained(
    MODEL_ID,
    low_cpu_mem_usage=True, # Экономия RAM при загрузке
    use_safetensors=True,
    torch_dtype=torch.float32 # Загружаем в FP32
).to("cpu")

model_to_cluster = copy.deepcopy(model_fp32_cpu_for_clustering)
model_to_cluster.to(cluster_device, dtype=cluster_dtype) # Переносим на нужное устройство и в нужный dtype
model_to_cluster.eval()
del model_fp32_cpu_for_clustering # Освобождаем память
gc.collect()
if cluster_device.type == 'cuda':
    torch.cuda.empty_cache()
print("Копия модели создана и перенесена.")


# 3. Функция для применения кластеризации к слою
def cluster_layer_weights(module, n_clusters):
    if not hasattr(module, 'weight') or module.weight is None:
        return 0, 0

    original_weights = module.weight.data.cpu().numpy().astype(np.float32) # K-Means лучше работает с FP32 CPU
    original_shape = original_weights.shape
    
    # K-means ожидает 2D массив [n_samples, n_features]
    # Если веса многомерные (например, для сверток, но мы здесь их не трогаем), их нужно решейпить.
    # Для nn.Linear веса уже 2D [out_features, in_features]
    weights_flat = original_weights.reshape(-1, 1) # K-means для одномерных данных (каждый вес - точка)

    if weights_flat.shape[0] < n_clusters:
        # print(f"  Количество весов ({weights_flat.shape[0]}) меньше числа кластеров ({n_clusters}). Пропуск слоя.")
        return 0,0 # Невозможно кластеризовать

    kmeans = KMeans(n_clusters=n_clusters, random_state=0, n_init='auto', algorithm='lloyd') # n_init='auto' для подавления warning
    kmeans.fit(weights_flat)
    
    centroids = kmeans.cluster_centers_
    labels = kmeans.labels_
    
    # Заменяем веса на значения центроидов
    weights_clustered_flat = centroids[labels].reshape(-1)
    weights_clustered = weights_clustered_flat.reshape(original_shape)
    
    # Обновляем веса слоя
    module.weight.data = torch.from_numpy(weights_clustered).to(module.weight.device, dtype=module.weight.dtype)
    
    # Подсчет теоретического размера: словарь центроидов + индексы
    # Размер центроидов: n_clusters * sizeof(float)
    # Размер индексов: num_weights * sizeof(индекс) (например, ceil(log2(n_clusters)) бит на индекс)
    size_of_float = 4 # байты для FP32 центроидов
    bits_per_index = np.ceil(np.log2(n_clusters)).astype(int)
    bytes_per_index = (bits_per_index + 7) // 8 # округляем до байтов

    theoretical_size_centroids = n_clusters * size_of_float
    theoretical_size_indices = weights_flat.shape[0] * bytes_per_index
    
    return theoretical_size_centroids, theoretical_size_indices


# 4. Применение кластеризации к выбранным слоям модели
print(f"Применение кластеризации (n_clusters={N_CLUSTERS}) к слоям типа {LAYERS_TO_CLUSTER}...")
total_theoretical_size_centroids = 0
total_theoretical_size_indices = 0
total_original_size_of_clustered_params = 0 # Для сравнения с теоретическим
bytes_per_original_param = 4 if model_to_cluster.dtype == torch.float32 else 2 # Используем dtype модели на устройстве

for module_name, module in model_to_cluster.named_modules():
    for layer_type in LAYERS_TO_CLUSTER:
        if isinstance(module, layer_type):
            # print(f"Кластеризация слоя: {module_name}")
            current_params = module.weight.data.numel()
            size_c, size_i = cluster_layer_weights(module, N_CLUSTERS)
            if size_c > 0: # Если кластеризация была применена
                 total_theoretical_size_centroids += size_c
                 total_theoretical_size_indices += size_i
                 total_original_size_of_clustered_params += current_params * bytes_per_original_param
            break # Переходим к следующему именованному модулю

print("Кластеризация весов завершена.")
theoretical_clustered_size_mb = (total_theoretical_size_centroids + total_theoretical_size_indices) / (1024 * 1024)
original_size_of_clustered_params_mb = total_original_size_of_clustered_params / (1024 * 1024)

# 5. Подготовка данных (если не были загружены ранее)
if 'quality_dataset' not in locals() or 'raw_audio_list_for_timing' not in locals():
    print("Перезагрузка датасетов...")
    quality_dataset = load_dataset(DATASET_ID, DATASET_NAME, split=f"{DATASET_SPLIT}[:{NUM_SAMPLES_FOR_QUALITY_TEST}]", trust_remote_code=True)
    quality_dataset = quality_dataset.cast_column("audio", Audio(sampling_rate=TARGET_SAMPLE_RATE))
    timing_dataset_raw = load_dataset(DATASET_ID, DATASET_NAME, split=f"{DATASET_SPLIT}[:{NUM_SAMPLES_FOR_TIME_TEST + NUM_WARMUP_RUNS}]", trust_remote_code=True)
    timing_dataset_raw = timing_dataset_raw.cast_column("audio", Audio(sampling_rate=TARGET_SAMPLE_RATE))
    raw_audio_list_for_timing = [sample["audio"]["array"] for sample in timing_dataset_raw]
    print("Датасеты перезагружены.")

# 6. Проведение замеров для кластеризованной модели
# forced_decoder_ids и processor должны быть доступны из первой ячейки

print("\nПроведение оценки качества (Кластеризованная модель)...")
clustered_wer, clustered_cer, _, _ = evaluate_quality_whisper(
    model_to_cluster, processor, quality_dataset, NUM_SAMPLES_FOR_QUALITY_TEST,
    cluster_device, cluster_dtype, forced_decoder_ids
)
print(f"Качество (WER) Кластеризованная модель: {clustered_wer if isinstance(clustered_wer, str) else clustered_wer:.4f}")
print(f"Качество (CER) Кластеризованная модель: {clustered_cer if isinstance(clustered_cer, str) else clustered_cer:.4f}")

print("\nПроведение замеров производительности (Кластеризованная модель)...")
clustered_time_ms, clustered_vram_mb, clustered_ram_mb = evaluate_performance_whisper(
    model_to_cluster, processor, raw_audio_list_for_timing, NUM_WARMUP_RUNS, NUM_SAMPLES_FOR_TIME_TEST,
    cluster_device, cluster_dtype, forced_decoder_ids
)

# Размер модели "на диске" (state_dict) после такой кластеризации не изменится,
# так как мы просто заменили значения весов. Реальное сжатие требует другого формата хранения.
# Но мы можем использовать теоретический размер.
num_params_total = sum(p.numel() for p in model_to_cluster.parameters()) # Все параметры
clustered_model_size_actual_storage_mb = (num_params_total * bytes_per_original_param) / (1024*1024)


# --- 7. Вывод результатов для Кластеризации ---
print("\n--- Результаты: Кластеризация весов (K-Means) ---")
print(f"Модель: {MODEL_ID}")
print(f"Метод: Кластеризация K-Means (n_clusters={N_CLUSTERS} для Linear слоев)")
print(f"Dtype: {cluster_dtype}")
print(f"Устройство: {cluster_device}")
print(f"Размер кластеризованных параметров (оригинальный, MB): {original_size_of_clustered_params_mb:.2f}")
print(f"Размер модели (теоретический после кластеризации, MB): {theoretical_clustered_size_mb:.2f} (только кластеризованные слои)")
print(f"Размер модели (фактическое хранение state_dict, MB): {clustered_model_size_actual_storage_mb:.2f}")
print(f"Время инференса ({cluster_device.type.upper()}, ms): {clustered_time_ms if isinstance(clustered_time_ms, str) else f'{clustered_time_ms:.2f}'}")
if cluster_device.type == 'cuda':
    print(f"Использование VRAM (MB): {clustered_vram_mb if isinstance(clustered_vram_mb, str) else f'{clustered_vram_mb:.2f}'}")
print(f"Использование RAM (MB): {clustered_ram_mb if isinstance(clustered_ram_mb, str) else f'{clustered_ram_mb:.2f}'}")
print(f"Качество (WER): {clustered_wer if isinstance(clustered_wer, str) else f'{clustered_wer:.4f}'}")
print(f"Качество (CER): {clustered_cer if isinstance(clustered_cer, str) else f'{clustered_cer:.4f}'}")

results_clustering = {
    "method": f"Clustering K-Means (k={N_CLUSTERS})",
    "model_id": MODEL_ID,
    "dtype": str(cluster_dtype),
    "device": str(cluster_device),
    "model_size_mb": f"{theoretical_clustered_size_mb:.2f} (theor.) / {clustered_model_size_actual_storage_mb:.2f} (actual st_dict)",
    "time_ms": f"{clustered_time_ms:.2f}" if isinstance(clustered_time_ms, (int, float)) else clustered_time_ms,
    "vram_mb": f"{clustered_vram_mb:.2f}" if isinstance(clustered_vram_mb, (int, float)) else clustered_vram_mb,
    "ram_mb": f"{clustered_ram_mb:.2f}" if isinstance(clustered_ram_mb, (int, float)) else clustered_ram_mb,
    "wer": f"{clustered_wer:.4f}" if isinstance(clustered_wer, (int, float)) else clustered_wer,
    "cer": f"{clustered_cer:.4f}" if isinstance(clustered_cer, (int, float)) else clustered_cer,
}
print("\nРезультаты для таблицы (Кластеризация):")
print(results_clustering)

# Очистка
del model_to_cluster
gc.collect()
if cluster_device.type == 'cuda':
    torch.cuda.empty_cache()

--- Кластеризация весов K-Means для openai/whisper-large-v3 ---
Создание копии модели openai/whisper-large-v3 для кластеризации на cuda с dtype torch.float16...
Копия модели создана и перенесена.
Применение кластеризации (n_clusters=32) к слоям типа [<class 'torch.nn.modules.linear.Linear'>]...
Кластеризация весов завершена.

Проведение оценки качества (Кластеризованная модель)...


Оценка качества (cuda, torch.float16): 100%|██████████| 50/50 [00:23<00:00,  2.11it/s]


Качество (WER) Кластеризованная модель: 0.1316
Качество (CER) Кластеризованная модель: 0.0464

Проведение замеров производительности (Кластеризованная модель)...


Замеры производительности (cuda, torch.float16): 100%|██████████| 10/10 [00:05<00:00,  1.77it/s]



--- Результаты: Кластеризация весов (K-Means) ---
Модель: openai/whisper-large-v3
Метод: Кластеризация K-Means (n_clusters=32 для Linear слоев)
Dtype: torch.float16
Устройство: cuda
Размер кластеризованных параметров (оригинальный, MB): 2926.63
Размер модели (теоретический после кластеризации, MB): 1463.38 (только кластеризованные слои)
Размер модели (фактическое хранение state_dict, MB): 2943.97
Время инференса (CUDA, ms): 557.09
Использование VRAM (MB): 3204.53
Использование RAM (MB): 12402.24
Качество (WER): 0.1316
Качество (CER): 0.0464

Результаты для таблицы (Кластеризация):
{'method': 'Clustering K-Means (k=32)', 'model_id': 'openai/whisper-large-v3', 'dtype': 'torch.float16', 'device': 'cuda', 'model_size_mb': '1463.38 (theor.) / 2943.97 (actual st_dict)', 'time_ms': '557.09', 'vram_mb': '3204.53', 'ram_mb': '12402.24', 'wer': '0.1316', 'cer': '0.0464'}
